## Scraping CNN

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
import concurrent.futures

In [7]:
jumlah_index = 1
threads_link = []
links = []
results = []
lock = threading.Lock()


In [8]:
def scrape_links(page_number):
    try:
        headers = {
            'User-Agent': 'Chrome/118.0.0.0'
        }
        url = f"https://www.cnnindonesia.com/search/?query=jokowi&page={page_number}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        div_badan = soup.find('div', {"class": "flex flex-col gap-5"})
        if div_badan:
            articles = div_badan.findAll('article', {"class": "flex-grow"})
            page_links = []
            for article in articles:
                link = article.find('a')['href']
                page_links.append(link)

            with lock:  # Lock to protect the shared 'links' list
                links.extend(page_links)

        else:
            print(f"Div 'flex flex-col gap-5' not found on page {page_number}")

        print(f"Scraped {len(page_links)} links from page {page_number}")
    except Exception as e:
        print(f"Error while scraping page {page_number}: {str(e)}")

In [9]:
if __name__ == "__main__":
    # Create a ThreadPoolExecutor with a number of threads matching CPU cores
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit scraping tasks
        for page_number in range(1, jumlah_index + 1):
            executor.submit(scrape_links, page_number)

    # Print the total number of links
    print("Total Links:", len(links))

Error while scraping page 1: 'NoneType' object is not subscriptable
Total Links: 0


Cetak dan simpan

In [5]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/cnn_sport_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')